In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44085
2,Huelva,Confirmados PDIA 14 días,1707
3,Huelva,Tasa PDIA 14 días,"332,6383069937837"
4,Huelva,Confirmados PDIA 7 días,640
5,Huelva,Tasa PDIA 7 dias,"124,71500672291833"
6,Huelva,Total Confirmados,44283
7,Huelva,Curados,40094
8,Huelva,Fallecidos,403


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44085.0


/tmp/ipykernel_4506/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12264.0


/tmp/ipykernel_4506/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4506/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4506/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4506/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 965 personas en los últimos 7 días 

Un positivo PCR cada 328 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44085.0,640.0,1707.0,513170.0,124.715007,332.638307,51.0
Huelva-Costa,26041.0,414.0,1124.0,289548.0,142.981475,388.191250,32.0
Condado-Campiña,13738.0,165.0,423.0,156231.0,105.612843,270.752924,15.0
Huelva (capital),12264.0,149.0,438.0,143837.0,103.589480,304.511357,13.0
Ayamonte,1969.0,32.0,132.0,21104.0,151.630023,625.473844,6.0
Cartaya,1833.0,54.0,134.0,20083.0,268.884131,667.230991,5.0
Moguer,1899.0,32.0,59.0,21867.0,146.339233,269.812960,5.0
Sierra de Huelva-Andévalo Central,3895.0,50.0,140.0,67391.0,74.193883,207.742874,4.0
Lepe,2931.0,53.0,125.0,27880.0,190.100430,448.350072,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Linares de la Sierra,11.0,2.0,6.0,262.0,763.358779,2290.076336,0.0
Almendro (El),57.0,11.0,18.0,840.0,1309.523810,2142.857143,0.0
Alájar,34.0,3.0,10.0,759.0,395.256917,1317.523057,0.0
Rosal de la Frontera,67.0,9.0,21.0,1697.0,530.347672,1237.477902,0.0
Cabezas Rubias,51.0,5.0,7.0,706.0,708.215297,991.501416,0.0
Escacena del Campo,225.0,9.0,17.0,2287.0,393.528640,743.331876,1.0
Cartaya,1833.0,54.0,134.0,20083.0,268.884131,667.230991,5.0
Isla Cristina,3007.0,49.0,141.0,21393.0,229.046884,659.094096,1.0
Ayamonte,1969.0,32.0,132.0,21104.0,151.630023,625.473844,6.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bollullos Par del Condado,1255.0,4.0,34.0,14387.0,27.802878,236.324460,0.0,0.117647
Cortegana,286.0,1.0,6.0,4602.0,21.729683,130.378096,0.0,0.166667
Ayamonte,1969.0,32.0,132.0,21104.0,151.630023,625.473844,6.0,0.242424
Cumbres Mayores,130.0,2.0,8.0,1749.0,114.351058,457.404231,1.0,0.250000
Alosno,225.0,1.0,4.0,3933.0,25.425884,101.703534,0.0,0.250000
Rociana del Condado,655.0,5.0,18.0,7939.0,62.980224,226.728807,0.0,0.277778
Palma del Condado (La),1399.0,14.0,49.0,10801.0,129.617628,453.661698,2.0,0.285714
Niebla,356.0,6.0,21.0,4116.0,145.772595,510.204082,1.0,0.285714
Alájar,34.0,3.0,10.0,759.0,395.256917,1317.523057,0.0,0.300000
